# AutoGen Grundläggande Exempel

I det här kodexemplet kommer du att använda [AutoGen](https://aka.ms/ai-agents/autogen) AI-ramverket för att skapa en grundläggande agent.

Målet med detta exempel är att visa dig stegen som vi senare kommer att använda i de ytterligare kodexemplen när vi implementerar de olika agentmönstren.


## Importera de nödvändiga Python-paketen


In [ ]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## Skapa klienten

I det här exemplet kommer vi att använda [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) för att få tillgång till LLM.

`model` är definierad som `gpt-4o-mini`. Prova att ändra modellen till en annan modell som finns tillgänglig på GitHub Models-marknaden för att se olika resultat.

Som ett snabbt test kommer vi att köra en enkel fråga - `Vad är huvudstaden i Frankrike`.


In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Definiera agenten

Nu när vi har konfigurerat `client` och bekräftat att den fungerar, låt oss skapa en `AssistantAgent`. Varje agent kan tilldelas:  
**namn** - Ett kort namn som är användbart för att referera till den i flöden med flera agenter.  
**model_client** - Den klient som du skapade i det tidigare steget.  
**verktyg** - Tillgängliga verktyg som agenten kan använda för att utföra en uppgift.  
**systemmeddelande** - Metaprompten som definierar uppgiften, beteendet och tonen för LLM.  

Du kan ändra systemmeddelandet för att se hur LLM reagerar. Vi kommer att gå igenom `verktyg` i Lektion #4.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## Kör agenten

Funktionen nedan kommer att köra agenten. Vi använder metoden `on_message` för att uppdatera agentens tillstånd med det nya meddelandet.

I det här fallet uppdaterar vi tillståndet med ett nytt meddelande från användaren som är `"Planera en fantastisk solig semester åt mig"`.

Du kan ändra meddelandets innehåll för att se hur LLM svarar annorlunda.


In [ ]:
from IPython.display import display, HTML


async def assistant_run():
    # Define the query
    user_query = "Plan me a great sunny vacation"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()


---

**Ansvarsfriskrivning**:  
Detta dokument har översatts med hjälp av AI-översättningstjänsten [Co-op Translator](https://github.com/Azure/co-op-translator). Även om vi strävar efter noggrannhet, bör du vara medveten om att automatiska översättningar kan innehålla fel eller inexaktheter. Det ursprungliga dokumentet på dess ursprungliga språk bör betraktas som den auktoritativa källan. För kritisk information rekommenderas professionell mänsklig översättning. Vi ansvarar inte för eventuella missförstånd eller feltolkningar som uppstår vid användning av denna översättning.
